In [ ]:
import pandas as pd
import numpy as np
import os as os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = '/content/drive/MyDrive/DATASET_AI_PROJECT/Brain_Tumor_Dataset/archive (2)/Training'
print(os.path.exists(dataset_path))

True


In [ ]:
import os
import pandas as pd

# Path to the dataset folder
dataset_path = '/content/drive/MyDrive/DATASET_AI_PROJECT/Brain_Tumor_Dataset/archive (2)/Training'

# Initialize a dictionary to hold image filenames for specified labels
data = {
    'glioma_tumor': [],
    'meningioma_tumor': [],
    'notumor': [],
    'pituitary_tumor': []
}

# Iterate through each folder in the dataset
for folder in data.keys():
    label_folder = os.path.join(dataset_path, folder.replace('_tumor', '') if 'tumor' in folder else 'notumour')
    if os.path.isdir(label_folder):  # Ensure it's a directory
        for image_filename in os.listdir(label_folder):
            if image_filename.endswith(('.jpg', '.png', '.jpeg')):  # Check for image files
                data[folder].append(image_filename)

# Create a DataFrame with specified labels as column headings
df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))

#df.to_csv('/content/drive/MyDrive/Processed_Medical_Data/Data.csv', index=False)

#print("CSV file created successfully!")

In [ ]:
import cv2  # library for reading images files
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
# Example directory structure
dataset_dir = '/content/drive/MyDrive/DATASET_AI_PROJECT/Brain_Tumor_Dataset/archive (2)/Training'

#When using a pretrained model for MRI brain tumor detection, you typically do not need to manually preprocess each image as extensively

#To have equal and compatible size as input into the model, the images were resized to 256x256 pixels
# Load pretrained ResNet50 model without top layers
input_shape = (224, 224,3)


img_path = '/content/drive/MyDrive/DATASET_AI_PROJECT/Brain_Tumor_Dataset/archive (2)/Training'

def normalize_intensity(img):
    """
    Normalize the intensity values of the input image to [0, 1].

    Parameters:
    - img: Input image as a numpy array.

    Returns:
    - Normalized image as a numpy array.
    """
    # Ensure the image is converted to float32 for safe division
    img = img.astype(np.float32)

    # Normalize the image to the range [0, 1]
    img_min = np.min(img)
    img_max = np.max(img)

    if img_max != img_min:
        normalized_img = (img - img_min) / (img_max - img_min)
    else:
        normalized_img = img / 255.0  # Handle the case when img_min == img_max (all pixels are the same)

    return normalized_img

def apply_histogram_equalization(img):
    """
    Apply histogram equalization to the input image.

    Parameters:
    - img: Input image as a numpy array (assuming grayscale).

    Returns:
    - Image with histogram equalization applied.
    """

    # Ensure image is in 8-bit unsigned integer format (CV_8UC1)
    if img.dtype != np.uint8:
        img = img.astype(np.uint8)

     # Ensure image is grayscale
    if len(img.shape) == 3 and img.shape[2] == 3:  # RGB image
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Apply histogram equalization using OpenCV
    equalized_img = cv2.equalizeHist(img)

    # Expand dimensions if the image was originally grayscale
    if len(img.shape) > 2 and img.shape[2] == 3:
        equalized_img = np.expand_dims(equalized_img, axis=2)
    # # Convert grayscale to RGB again
    # if len(img.shape) == 2 or img.shape[2] == 1:
    #     img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    return equalized_img


def custom_preprocessing(img):
    # Apply intensity normalization
    normalized_img = normalize_intensity(img)

    # Apply histogram equalization (assuming grayscale images)
    #equalized_img = apply_histogram_equalization(normalized_img)

    #convert to rbg
    return normalized_img


from tensorflow.keras.applications.resnet import preprocess_input
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Flow from directory with classes automatically inferred from subfolders
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Adjust class_mode based on your dataset (binary/multi-class)
    shuffle=True
)
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



images, labels = next(train_generator)


Found 4850 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [ ]:
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}
print("Class Labels Mapping:", class_labels)

Class Labels Mapping: {0: 'glioma', 1: 'meningioma', 2: 'notumor', 3: 'pituitary'}


In [ ]:
#preprocessed_dir='/content/drive/MyDrive/DATASET_AI_PROJECT/Brain_Tumor_Dataset/Processed_Medical_Data/Training'

EXTRACTING FEATURES


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

# Load ResNet50 model without top layer (classifier)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a model that outputs features
model = Model(inputs=base_model.input, outputs=base_model.output)

# Extract features for training data
def extract_features(generator):
    features = []
    labels = []
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
    return np.vstack(features), np.vstack(labels)

train_features, train_labels = extract_features(train_generator)
validation_features, validation_labels = extract_features(validation_generator)

1/1 [==============================] - 0s 46ms/step


TRAINING MODEL WITH CLASSIFICATION ANN MODEL



In [ ]:

from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential

# Define a new model for classification
classification_model = Sequential([
    Flatten(input_shape=train_features.shape[1:]),  # Flatten the features
    Dense(256, activation='relu'),
    Dense(train_labels.shape[1], activation='softmax')  # Output layer for classification
])

classification_model.compile(optimizer='adam',
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

# Train the classification model
classification_model.fit(train_features, train_labels,
                         epochs=10,
                         batch_size=32,
                         validation_data=(validation_features, validation_labels))




EVALUATE AND USE YOUR MODEL


In [ ]:
loss, accuracy = classification_model.evaluate(validation_features, validation_labels)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

# Use the model for predictions
predictions = classification_model.predict(validation_features)